EE541 project

install all the packages

In [1]:
!pip install -q accelerate peft transformers trl

import all the tools

In [2]:
!pip install git+https://github.com/mit-han-lab/smoothquant.git


  Cloning https://github.com/mit-han-lab/smoothquant.git to /tmp/pip-req-build-ffjkl5e3
  Running command git clone --filter=blob:none --quiet https://github.com/mit-han-lab/smoothquant.git /tmp/pip-req-build-ffjkl5e3
  Resolved https://github.com/mit-han-lab/smoothquant.git to commit c61476d728e42ae0d8a35e7e78494edcac3237b5
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install --upgrade transformers


  Using cached transformers-4.47.0-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.47.0-py3-none-any.whl (10.1 MB)
Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0


In [3]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
from smoothquant.smooth import smooth_lm
from smoothquant.fake_quant import quantize_llama_like

In [4]:
from datasets import load_dataset

In [30]:
!pip install --upgrade accelerate


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [6]:
from huggingface_hub import login

In [ ]:
login()

In [ ]:
import os
print(os.listdir("."))

['.config', 'llama-2-7b.pt', 'opt-1.3b.pt', '.ipynb_checkpoints', 'sample_data']


In [ ]:
import os

# Verify file existence
act_scales_path = "llama-2-7b.pt"
if os.path.exists(act_scales_path):
    print(f"File {act_scales_path} exists.")
else:
    print(f"File {act_scales_path} is missing.")

File llama-2-7b.pt exists.


In [7]:
# Define the model name
base_model_name = "facebook/opt-1.3b"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
model16 = AutoModelForCausalLM.from_pretrained(base_model_name, torch_dtype="auto")
#had login problem with llama2, so decided to use another similar model

# base_model_name = "meta-llama/Llama-2-7b-hf"
# model16 = LlamaForCausalLM.from_pretrained(
#     base_model_name,
#     torch_dtype=torch.float16
#     #device_map="auto"
# )

# # Load tokenizer
# tokenizer = LlamaTokenizer.from_pretrained(base_model_name)



# dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
# # Prepare tokenized inputs for scale computation
# tokenized_dataset = tokenizer(
#     dataset["text"], return_tensors="pt", padding=True, truncation=True
# )
# # Compute activation scales
# act_scales = compute_activation_scales(
#     model=model_fp16,
#     dataloader=tokenized_dataset,
#     num_samples=100  # Adjust based on dataset size
# )
# # Save scales to file
# torch.save(act_scales, "llama-2-7b2.pt")



# Load the SmoothQuant activation scales
act_scales_path = "opt-1.3b.pt"
#act_scales_path = "llama-2-7b.pt"
act_scales = torch.load(act_scales_path)

# Apply smoothing (adjust the alpha parameter as needed)
smooth_alpha = 0.85
smooth_lm(model16, act_scales, smooth_alpha)


# Save this smoothed model for later use:
model_smoothquant_w8a8 = quantize_llama_like(model16)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
<ipython-input-7-e2277f3fe2bf>:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the 

perplexity score evaluation code

In [8]:
import tqdm
import torch.nn as nn

In [ ]:
#perplexity code from online source
class Evaluator:
    def __init__(self, dataset, tokenizer, device, n_samples=40):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.device = device

        self.dataset = tokenizer(
            "\n\n".join(dataset["text"]), return_tensors="pt"
        ).input_ids.to(device)

        self.n_samples = n_samples

    @torch.no_grad()
    def evaluate(self, model):
        model.eval()
        nlls = []
        for i in tqdm.tqdm(range(self.n_samples), desc="Evaluating..."):
            batch = self.dataset[:, (i * 2048) : ((i + 1) * 2048)].to(model.device)
            with torch.no_grad():
                lm_logits = model(batch).logits
            shift_logits = lm_logits[:, :-1, :].contiguous().float()
            shift_labels = self.dataset[:, (i * 2048) : ((i + 1) * 2048)][:, 1:]
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(
                shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1)
            )
            neg_log_likelihood = loss.float() * 2048
            nlls.append(neg_log_likelihood)

        return torch.exp(torch.stack(nlls).sum() / (self.n_samples * 2048))

In [ ]:
dataset = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
evaluator = Evaluator(dataset, tokenizer,"cpu")

In [ ]:
ppl_in8 = evaluator.evaluate(model_smoothquant_w8a8) #see the performance of the quantized model
print(f"Original model (in8) perplexity: {model_smoothquant_w8a8}")

In [ ]:
!pip install trl --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 47.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
  Attempting uninstall: trl
    Found existing installation: trl 0.4.7
    Uninstalling trl-0.4.7:
      Successfully uninstalled trl-0.4.7


In [9]:
from transformers import TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [10]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

now we can start fine tuning

In [11]:
# QLoRA configuration
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [12]:
dataset_name = "mlabonne/guanaco-llama2-1k"
dataset = load_dataset(dataset_name, split="train")

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,  # Ensure FP16 training is enabled for efficiency
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    group_by_length=True,
    report_to="tensorboard",
)

# Trainer
trainer = SFTTrainer(
    model=model_smoothquant_w8a8,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_args,
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:412: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(


we can stop here if we don't want a standalone model, we can have dynamic inference, but if we want a standalone model we can merge the original with this

In [ ]:
# finetunemodel = "opt-finetune"
# trainer.model.save_pretrained(new_model)
# # Reload model in FP16 and merge it with LoRA weights
# base_model = AutoModelForCausalLM.from_pretrained(
#     base_model_name,
#     low_cpu_mem_usage=True,
#     return_dict=True,
#     torch_dtype=torch.float16,
#     # device_map=device_map,
# )
# model = PeftModel.from_pretrained(base_model, finetunemodel)
# model = model.merge_and_unload()

# # Reload tokenizer to save it
# tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"